In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
bkgfiles = [
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_bEnriched_HT1000to1500_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_bEnriched_HT100to200_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_bEnriched_HT1500to2000_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_bEnriched_HT2000toInf_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_bEnriched_HT200to300_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_bEnriched_HT300to500_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_bEnriched_HT500to700_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_bEnriched_HT700to1000_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_HT1000to1500_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_HT100to200_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_HT1500to2000_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_HT2000toInf_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_HT200to300_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_HT300to500_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_HT500to700_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_Autumn18//QCD/QCD_HT700to1000_BGenFilter_TuneCP5_13TeV-madgraph-pythia8/ntuple.root",
"/store/user/ekoenig/8BAnalysis/NTuples/2018/preselection/t8btag_minmass//Run2_UL/RunIISummer20UL18NanoAODv9/TTJets/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/ntuple_training.root",
]

In [6]:
bkgtree = Tree(bkgfiles, altfile='training_{base}')

 59%|█████▉    | 10/17 [00:02<00:01,  3.87it/s]

[WARNING] skipping None, was not found.


100%|██████████| 17/17 [00:05<00:00,  3.30it/s]


In [10]:
ak.sum(bkgtree.dataset_norm_abs_scale)

821.2709351319407

In [12]:
sigtree = Tree('/store/user/ekoenig/8BAnalysis/NTuples/2018/training/training_5M//NMSSM_XYY_YToHH_8b/NMSSM_XYY_YToHH_8b_MX_1000_MY_450/ntuple.root', altfile='fully_res_{base}')

100%|██████████| 1/1 [00:00<00:00,  3.05it/s]


In [13]:
ak.sum(sigtree.dataset_norm_abs_scale)

821.2709351319411